In [ ]:
#Helps reading http.
import requests
#Finds pathnames
import glob
#manipulation files and path names
import os

# Pulling and analyising "NR Abstimmungsdatenbank"

Quelle: [Parlament.ch](https://www.parlament.ch/de/ratsbetrieb/abstimmungen/abstimmung-nr-xls)

In [ ]:
requests.get('https://www.parlament.ch/centers/documents/_layouts/15/DocIdRedir.aspx?ID=DOCID-1-8569')

In [ ]:
#Testing the download
dls = "https://www.parlament.ch/centers/documents/_layouts/15/DocIdRedir.aspx?ID=DOCID-1-8569"
resp = requests.get(dls)
with open('data2/test.xls', 'wb') as output:
    output.write(resp.content)

In [ ]:
#Iterating trough files and saving them off. Max: 8600
for number in range(8300, 8320):
    dls = "https://www.parlament.ch/centers/documents/_layouts/15/DocIdRedir.aspx?ID=DOCID-1-" + str(number)
    resp = requests.get(dls)
    with open('data2/' + str(number) + '.xls', 'wb') as output:
        output.write(resp.content)

In [ ]:
#Now getting all the files names
list_of_files = []
for name in glob.glob('data2/*'):
    name = name.split('/')[-1]
    list_of_files.append(name)

In [ ]:
#And deleting the ones that weren't .xls files
for data in list_of_files:
    try:
        df = pd.read_excel('data2/' + data, header=8)
    except:
        'XLRDError'
        os.remove('data2/' + data)

# Molding the DF

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('data/5005-2016-herbstsession-d.xlsx')
df.head(10)

In [ ]:
#Pulling out the whole df, using only the headers
df = pd.read_excel('data/5005-2016-herbstsession-d.xlsx', header=8)

In [ ]:
df.head(2)

In [ ]:
#Pulling out name of the politicians
df_name = pd.read_excel('data/5005-2016-herbstsession-d.xlsx', header=2)

In [ ]:
#Replacing the number with the name
pol_names = list(df_name.iloc[[1]])

In [ ]:
#pulling out first section
df_first_11 = df.iloc[:, :11]

In [ ]:
#Renaming the politician columns from numbers to names
df_politicians = df.iloc[:, 11:]
df_politicians.columns = pol_names
df_politicians = df_politicians.iloc[:, :-7]

In [ ]:
#results
df_result = df.iloc[:, -7:]

In [ ]:
#Recreating the file
df_molded = pd.concat([df_first_11, df_politicians, df_result], axis=1, join_axes=[df_first_11.index])

# Working out who has best rating

In [ ]:
#function to translate Decision to German
def translate(x):
    if x == 'Yes':
        return 'Ja'
    else:
        return 'Nein'

In [ ]:
df_molded['Decision'] = df_molded['Decision'].apply(translate)

In [ ]:
#Checking on politician
Zuberbühler_david = list(df_molded['Zuberbühler David'])

In [ ]:
decision_list = list(df_molded['Decision'])

In [ ]:
count = 0
for x, y in zip(Zuberbühler_david, decision_list):
    if x == y:
        count = count + 1

In [ ]:
#Creating second list only with last name
last_pol_names = []
for last_name in pol_names:
    last_name = last_name.split(' ')
    last_name = last_name[0]
    last_pol_names.append(last_name)

In [ ]:
last_pol_names.pop(0)

In [ ]:
pol_names.pop(0)

In [ ]:
pol_names = pol_names[:-7]

In [ ]:
#Iterating through the two lists to work out values of them all
decision_list = list(df_molded['Decision'])
pol_decision_count = []

for name, last_name in zip(pol_names, last_pol_names):
    #print(name, last_name)
    count = 0
    #print(count)
    try:
        last_name = list(df_molded[name])
    except:
        'KeyError'
    #print(last_name)
    try:
        for x, y in zip(last_name, decision_list):
            if x == y:
                count = count + 1
            
            pol_decisions = {'Name': name,
                        'Mit Mehrheit': count}
        pol_decision_count.append(pol_decisions)
    except:
        'KeyError'
    #print(pol_decision_count)

In [ ]:
#Making DF
df_count = pd.DataFrame(pol_decision_count)

In [ ]:
#Top 5
df_count.sort_values(by='Mit Mehrheit', ascending=False).head()

In [ ]:
#Bottom 5
df_count.sort_values(by='Mit Mehrheit', ascending=True).head()

In [ ]:
df_count.to_csv('Mit_der_Mehrheit_Herbstsession_2016.csv')